In [13]:
#1.Pulls delivery restaurants details from Zomato
#2.Need to add the start url of a locality
#3.set the popular_locality

In [155]:
import requests
from bs4 import BeautifulSoup
import re
import pandas
from collections import OrderedDict

In [156]:
startUrl = "https://www.zomato.com/bangalore/restaurants"

In [157]:
#Used headers/agent because the request was timed out and asking for an agent. 
#Using following code we can fake the agent.

def getSoup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    response = requests.get(url,headers=headers)
    
    content = response.content
    soup = BeautifulSoup(content,"html.parser")
    print(url)
    return soup

In [158]:
#max pages
def getMaxPages():
    soup = getSoup(startUrl)
    pages = []
    pagesTag = soup.find("div",attrs={"class":"col-l-4 mtop pagination-number"})
    pagesTag_temp = pagesTag.find_all("b",recursive=True)
    
    for p in pagesTag_temp:
        pages.append(p.get_text())
        
    return int(max(pages))


In [159]:

list_dataframes = []
currentpage = 1
maxPages = getMaxPages()
print(maxPages)
while currentpage <= maxPages:
    
    pageUrl = startUrl+"?page="+str(currentpage)
    soup = getSoup(pageUrl)
    list_rest = soup.find_all("div",attrs={"class": "col-s-12"})
    
    for rest in list_rest:
        dataframe = OrderedDict() #dict

        #restaurant link
        link = rest.find("a",attrs={"class": "result-title hover_feedback zred bold ln24 fontsize0"})
        dataframe["restaurant_link"] = link['href']
        
        soup = getSoup(link['href'])
        
        #Res Id
        res_id = soup.find("head")
        scripts = res_id.select('script')
        for script in scripts:
            if 'window.RES_ID' in script.text:
                ids = re.findall('\d+', script.contents[0])
                dataframe["restaurant_ID"] = ids[0]
        
        #restaurant title
        top_header_title = (soup.find("h1",attrs={"class":"ui res-name mb0 header nowrap"}))
        dataframe["restaurant_name"] = top_header_title.find("a").get_text().strip()

        #locality
        loc_title = (soup.find("div",attrs={"class":"mb5 pt5 clear"}))
        dataframe["locality"] = loc_title.find("a").get_text().strip()

        #category
        res_category_tag = loc_title.find("span",attrs={"class":"res-info-estabs grey-text fontsize3"})
        if res_category_tag is not None:
            res_category = res_category_tag.find("a")
            if res_category is not None:
                dataframe["restaurant_category"] = res_category_tag.find("a").get_text().strip()
            else:
                dataframe["restaurant_category"] = res_category_tag.get_text().strip()#Delivery
        
        #zomato gold
        goldTag = soup.find("div",attrs={"class":"red_res_tag"})
        if goldTag is not None:
            goldTag_temp = goldTag.find("img")
            if goldTag_temp is not None:
                dataframe["zomato_gold"] = "zomato gold" 

        #discounts
        discountTag = soup.find_all("div",attrs={"class": "fontsize flex flex-center-v"})
        for child in discountTag:
            discountTag_temp = child.find("div",attrs={"class":"pr5"})
            dataframe["discounts"] = discountTag_temp.getText()
            
        #photos
        photosTag = soup.find_all("div",attrs={"respageMenuContainer"})
        for photo in photosTag:
            children = photo.findChildren("a",attrs={"item respageMenu-item photosTab restaurant-tab-item-jumbo-track"})
            for child in children:
                reObj = re.compile('\d+')
                dataframe["photos_taken"] = reObj.findall(child.text.strip())[0]

        #rating
        top_header_rating = (soup.find("div",attrs={"class":"col-l-4 tac left"}))
        rating = top_header_rating.find("div",attrs={"class":"rating_hover_popup res-rating pos-relative clearfix mb5"})

        if rating is not None:
            dataframe["rating"] = rating.get_text().replace('\n',' ').strip()
        else:
            dataframe["rating"] = "NA"

        #votes
        top_header_votes = top_header_rating.find("span",attrs={"class":"mt2 mb0 rating-votes-div rrw-votes grey-text fontsize5 ta-right"})
        if top_header_votes is not None:
            reObj = re.compile('\d+')
            dataframe["votes"] = reObj.findall(top_header_votes.get_text())[0]
        else:
            dataframe["votes"] = "NA"

        #cuisines
        info_left = (soup.find("div",attrs={"class":"col-l-1by3 pl0 pr20"}))
        cuisine_info = info_left.find("div",attrs={"class":"res-info-cuisines clearfix"})
        dataframe["cuisines"] = cuisine_info.get_text()

        #Approx cost
        approxCost_tag = info_left.find("div",attrs={"class":"res-info-detail"})
        if approxCost_tag is not None:
            approx_cost = approxCost_tag.find("span",attrs={"tabindex":"0"})
        if approx_cost is not None:
            reObj = re.compile('\d+')
            dataframe["approx_cost_for_2"] = reObj.findall(approx_cost.get_text().replace(',','').strip())[0]
        else:
            dataframe["approx_cost_for_2"] = "NA"
        
        #Opening timings
        info_centre = (soup.find("div",attrs={"class":"col-l-1by3 pl20 pr20"}))
        if info_centre is not None:
            openingtimes = info_centre.find("div",attrs={"class":"res-info-timings"})
        if openingtimes is not None:
            timings = openingtimes.find("div",attrs={"class":"medium"})
            dataframe["opening_timings"] = re.findall(r'\s(\d*\:\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours)|\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours))',timings.get_text())
        
        #address
        addressTag = info_centre.find("div",attrs={"class":"mbot0"})
        address = addressTag.find("span")
        if addressTag and address is not None:
            dataframe["address"] = address.get_text()
        else:
            dataframe["address"] = "NA"
        
        #lat-long
        lat_longTag = soup.find("div",attrs={"class":"resmap pos-relative mt5 mb5"})
        if lat_longTag is not None:
            lat_long_scripts = lat_longTag.select('script')
            for script in lat_long_scripts:
                reObj = re.compile('\d*\.\d*')
                lat_lon = reObj.findall(script.contents[0])
                
                if lat_lon is not None:
                    dataframe["latitude"] = lat_lon[0]
                    dataframe["longitude"] = lat_lon[1]
                else:
                    dataframe["longitude"] = dataframe["latitude"] = "NA"

        #more info
        more_info = (soup.find_all("div",attrs={"class":"res-info-highlights"}))
        for child in more_info:
            infos = child.find_all("div",attrs={"class":"res-info-feature-text"})
            infostring = ''
            for info in infos:
                infostring += info.get_text()+', '
            dataframe["more_info"] = infostring
            
        #Featured in Collections
        featured_in_tag = soup.find("div",attrs={"class":"mbot0 ptop0"})
        if featured_in_tag is not None:
            featured_in = featured_in_tag.find("div",attrs={"class":"resbox__main--row res-info-group clearfix"})
        if featured_in is not None:
            features = featured_in.find_all("a")
        if features is not None: 
            dataframe["featured_in"] = [ftr.get_text().strip() for ftr in features]
        else:
            dataframe["featured_in"] = "NA"
            
        #known for
        knownForTag = soup.find_all("div",attrs={"class":"mbot0 ptop0"})
        for child in knownForTag:
            known_for_list = child.find_all("div",attrs={"class":"res-info-known-for-text mr5"})
            for known_for in known_for_list:
                dataframe["known_for"] = known_for.get_text().strip()

        #loved here food
        mostlovedTags = soup.find_all("div",attrs={"class":"rv_highlights__wrapper mtop0"})
        for child in mostlovedTags:
            j = child.find_all("div",attrs={"class":"rv_highlights__section pr10"})
        for op in j:
            mostlovedcategory = op.find("div",attrs={"class":"grey-text"})

            loved_here = op.find("div",attrs={"class":"rv_highlights__score_bar mt5 mb5"})
            blockbg = loved_here.find_all("div",attrs={"class":"block bd-txt-bg"})

            mostLoved = op.find("div",attrs={"class":"fontsize13 ln18"})
            if mostLoved is not None:
                item = mostLoved.find_all("span")
                if item is not None:
                    temp = []
                    temp.append(str(5-len(blockbg))+"/5")
                    for i in item:
                        temp.append(i.get_text().replace(',','').strip())
                    dataframe["most_liked_"+mostlovedcategory.get_text()] = temp 

        ##reviews
        review_link = link['href']+"/reviews"
        review_soup = getSoup(review_link)

        review_header = (review_soup.find_all("div",attrs={"class":"res-reviews-container res-reviews-area"}))
        review_list = []
        for i in review_header:
            rev_children = i.findChildren("div",attrs={"rev-text mbot0"})
            for child in rev_children:
                reviewtxt = child.text.replace('Rated\xa0\n','').strip()
                review_list.append(reviewtxt.replace('\n',''))

        dataframe["reviews"] = review_list
        
        list_dataframes.append(dataframe)

    currentpage += 1
else:
    currentpage = 1
        



https://www.zomato.com/bangalore/restaurants
929
https://www.zomato.com/bangalore/restaurants?page=751
https://www.zomato.com/bangalore/orbis-market-kitchen-whitefield
https://www.zomato.com/bangalore/orbis-market-kitchen-whitefield/reviews
https://www.zomato.com/bangalore/nys-kitchen-1-whitefield-bangalore
https://www.zomato.com/bangalore/nys-kitchen-1-whitefield-bangalore/reviews
https://www.zomato.com/bangalore/village-cafe-hebbal
https://www.zomato.com/bangalore/village-cafe-hebbal/reviews
https://www.zomato.com/bangalore/ambur-hot-dum-biriyani-kabab-center-thippasandra-bangalore
https://www.zomato.com/bangalore/ambur-hot-dum-biriyani-kabab-center-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/raspberry-international-airport
https://www.zomato.com/bangalore/raspberry-international-airport/reviews
https://www.zomato.com/bangalore/food-time-now-rammurthy-nagar-bangalore
https://www.zomato.com/bangalore/food-time-now-rammurthy-nagar-bangalore/reviews
https://www.zomat

https://www.zomato.com/bangalore/restaurants?page=755
https://www.zomato.com/bangalore/balaji-biriyani-corner-hbr-layout
https://www.zomato.com/bangalore/balaji-biriyani-corner-hbr-layout/reviews
https://www.zomato.com/bangalore/café-delight-marathahalli
https://www.zomato.com/bangalore/café-delight-marathahalli/reviews
https://www.zomato.com/bangalore/green-county-family-restaurant-mysore-road
https://www.zomato.com/bangalore/green-county-family-restaurant-mysore-road/reviews
https://www.zomato.com/bangalore/simple-thindies-banashankari
https://www.zomato.com/bangalore/simple-thindies-banashankari/reviews
https://www.zomato.com/bangalore/bengalore-dum-biryani-yeshwantpur
https://www.zomato.com/bangalore/bengalore-dum-biryani-yeshwantpur/reviews
https://www.zomato.com/bangalore/ashirwad-grand-jalahalli
https://www.zomato.com/bangalore/ashirwad-grand-jalahalli/reviews
https://www.zomato.com/bangalore/shree-krishna-foods-nagawara
https://www.zomato.com/bangalore/shree-krishna-foods-nagaw

https://www.zomato.com/bangalore/café-s-koramangala-4th-block/reviews
https://www.zomato.com/dessertstree
https://www.zomato.com/dessertstree/reviews
https://www.zomato.com/bangalore/restaurants?page=759
https://www.zomato.com/bangalore/café-s-koramangala-4th-block
https://www.zomato.com/bangalore/café-s-koramangala-4th-block/reviews
https://www.zomato.com/dessertstree
https://www.zomato.com/dessertstree/reviews
https://www.zomato.com/bangalore/ritikas-kkitchen-jp-nagar-bangalore
https://www.zomato.com/bangalore/ritikas-kkitchen-jp-nagar-bangalore/reviews
https://www.zomato.com/bangalore/sri-annapurna-andhra-style-cv-raman-nagar
https://www.zomato.com/bangalore/sri-annapurna-andhra-style-cv-raman-nagar/reviews
https://www.zomato.com/bangalore/aishwarya-whitefield
https://www.zomato.com/bangalore/aishwarya-whitefield/reviews
https://www.zomato.com/bangalore/mayuri-delux-restaurant-whitefield
https://www.zomato.com/bangalore/mayuri-delux-restaurant-whitefield/reviews
https://www.zomato.c

https://www.zomato.com/bangalore/hotel-amulya-1-electronic-city/reviews
https://www.zomato.com/bangalore/reddy-dhaba-bommasandra
https://www.zomato.com/bangalore/reddy-dhaba-bommasandra/reviews
https://www.zomato.com/bangalore/sk-green-gardens-bommasandra
https://www.zomato.com/bangalore/sk-green-gardens-bommasandra/reviews
https://www.zomato.com/bangalore/restaurants?page=763
https://www.zomato.com/bangalore/sk-green-gardens-bommasandra
https://www.zomato.com/bangalore/sk-green-gardens-bommasandra/reviews
https://www.zomato.com/bangalore/hotel-sri-chiraag-sagar-bommasandra
https://www.zomato.com/bangalore/hotel-sri-chiraag-sagar-bommasandra/reviews
https://www.zomato.com/bangalore/new-shanthi-sagar-banaswadi
https://www.zomato.com/bangalore/new-shanthi-sagar-banaswadi/reviews
https://www.zomato.com/bangalore/tandoori-platter-residency-road
https://www.zomato.com/bangalore/tandoori-platter-residency-road/reviews
https://www.zomato.com/bangalore/sri-upahara-hsr
https://www.zomato.com/ba

https://www.zomato.com/bangalore/mongolian-kitchen-btm/reviews
https://www.zomato.com/bangalore/spice-taj-btm
https://www.zomato.com/bangalore/spice-taj-btm/reviews
https://www.zomato.com/bangalore/settys-banashankari
https://www.zomato.com/bangalore/settys-banashankari/reviews
https://www.zomato.com/bangalore/restaurants?page=767
https://www.zomato.com/bangalore/red-wings-rammurthy-nagar
https://www.zomato.com/bangalore/red-wings-rammurthy-nagar/reviews
https://www.zomato.com/bangalore/parivaar-koramangala-7th-block
https://www.zomato.com/bangalore/parivaar-koramangala-7th-block/reviews
https://www.zomato.com/bangalore/lotus-bell-new-bel-road
https://www.zomato.com/bangalore/lotus-bell-new-bel-road/reviews
https://www.zomato.com/bangalore/sv-indian-restaurant-hennur
https://www.zomato.com/bangalore/sv-indian-restaurant-hennur/reviews
https://www.zomato.com/bangalore/delhis-essence-electronic-city
https://www.zomato.com/bangalore/delhis-essence-electronic-city/reviews
https://www.zomat

...

https://www.zomato.com/bangalore/mad-belgian-waffles-electronic-city-bangalore
https://www.zomato.com/bangalore/mad-belgian-waffles-electronic-city-bangalore/reviews
https://www.zomato.com/bangalore/shankar-bhel-house-frazer-town
https://www.zomato.com/bangalore/shankar-bhel-house-frazer-town/reviews
https://www.zomato.com/bangalore/ishta-upahar-ulsoor
https://www.zomato.com/bangalore/ishta-upahar-ulsoor/reviews
https://www.zomato.com/bangalore/kairali-btm
https://www.zomato.com/bangalore/kairali-btm/reviews
https://www.zomato.com/bangalore/tasty-food-1-yeshwantpur-bangalore
https://www.zomato.com/bangalore/tasty-food-1-yeshwantpur-bangalore/reviews
https://www.zomato.com/bangalore/new-biriyani-house-rt-nagar-bangalore
https://www.zomato.com/bangalore/new-biriyani-house-rt-nagar-bangalore/reviews
https://www.zomato.com/bangalore/nava-ruchi-marathahalli-bangalore
https://www.zomato.com/bangalore/nava-ruchi-marathahalli-bangalore/reviews
https://www.zomato.com/bangalore/tandoor-hennur-ba

https://www.zomato.com/bangalore/ny-pizza-waffles-electronic-city-bangalore/reviews
https://www.zomato.com/bangalore/udupi-savi-thindi-sarjapur-road-bangalore
https://www.zomato.com/bangalore/udupi-savi-thindi-sarjapur-road-bangalore/reviews
https://www.zomato.com/bangalore/paratha-king-bellandur
https://www.zomato.com/bangalore/paratha-king-bellandur/reviews
https://www.zomato.com/bangalore/angel-restaurant-bannerghatta-road
https://www.zomato.com/bangalore/angel-restaurant-bannerghatta-road/reviews
https://www.zomato.com/bangalore/gonguras-multicuisine-family-restaurant-rajarajeshwari-nagar-bangalore
https://www.zomato.com/bangalore/gonguras-multicuisine-family-restaurant-rajarajeshwari-nagar-bangalore/reviews
https://www.zomato.com/bangalore/damodar-seth-marathahalli
https://www.zomato.com/bangalore/damodar-seth-marathahalli/reviews
https://www.zomato.com/bangalore/southern-taste-electronic-city-bangalore
https://www.zomato.com/bangalore/southern-taste-electronic-city-bangalore/revi

https://www.zomato.com/bangalore/archanas-cafe-btm-bangalore
https://www.zomato.com/bangalore/archanas-cafe-btm-bangalore/reviews
https://www.zomato.com/bangalore/sabzi-aur-roti-btm-bangalore
https://www.zomato.com/bangalore/sabzi-aur-roti-btm-bangalore/reviews
https://www.zomato.com/bangalore/s-k-tiffin-room-city-market-bangalore
https://www.zomato.com/bangalore/s-k-tiffin-room-city-market-bangalore/reviews
https://www.zomato.com/bangalore/bangalore-chats-thippasandra-bangalore
https://www.zomato.com/bangalore/bangalore-chats-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/restaurants?page=920
https://www.zomato.com/bangalore/republic-shakes-and-snacks-electronic-city-bangalore
https://www.zomato.com/bangalore/republic-shakes-and-snacks-electronic-city-bangalore/reviews
https://www.zomato.com/bangalore/sultans-garam-masala-nagawara-bangalore
https://www.zomato.com/bangalore/sultans-garam-masala-nagawara-bangalore/reviews
https://www.zomato.com/bangalore/alkausar-food-c

https://www.zomato.com/bangalore/sri-krishna-sannidhi-grand-kammanahalli-bangalore
https://www.zomato.com/bangalore/sri-krishna-sannidhi-grand-kammanahalli-bangalore/reviews
https://www.zomato.com/bangalore/taste-of-tibet-1-nagawara-bangalore
https://www.zomato.com/bangalore/taste-of-tibet-1-nagawara-bangalore/reviews
https://www.zomato.com/bangalore/tea-positive-marathahalli-bangalore
https://www.zomato.com/bangalore/tea-positive-marathahalli-bangalore/reviews
https://www.zomato.com/bangalore/nanma-family-restaurant-hennur-bangalore
https://www.zomato.com/bangalore/nanma-family-restaurant-hennur-bangalore/reviews
https://www.zomato.com/bangalore/the-italian-corner-ejipura-bangalore
https://www.zomato.com/bangalore/the-italian-corner-ejipura-bangalore/reviews
https://www.zomato.com/bangalore/hotel-taj-palace-bellandur-bangalore
https://www.zomato.com/bangalore/hotel-taj-palace-bellandur-bangalore/reviews
https://www.zomato.com/bangalore/wow-shakes-btm-bangalore
https://www.zomato.com/b

https://www.zomato.com/bangalore/nammuru-mangaluru-sahakara-nagar-bangalore/reviews
https://www.zomato.com/bangalore/the-ibrahims-sanjay-nagar-bangalore
https://www.zomato.com/bangalore/the-ibrahims-sanjay-nagar-bangalore/reviews
https://www.zomato.com/bangalore/food-factory-new-bel-road-bangalore
https://www.zomato.com/bangalore/food-factory-new-bel-road-bangalore/reviews
https://www.zomato.com/bangalore/food-me-btm-bangalore
https://www.zomato.com/bangalore/food-me-btm-bangalore/reviews
https://www.zomato.com/bangalore/restaurants?page=927
https://www.zomato.com/bangalore/the-ibrahims-sanjay-nagar-bangalore
https://www.zomato.com/bangalore/the-ibrahims-sanjay-nagar-bangalore/reviews
https://www.zomato.com/bangalore/food-factory-new-bel-road-bangalore
https://www.zomato.com/bangalore/food-factory-new-bel-road-bangalore/reviews
https://www.zomato.com/bangalore/food-me-btm-bangalore
https://www.zomato.com/bangalore/food-me-btm-bangalore/reviews
https://www.zomato.com/bangalore/buffalo-w

In [160]:
df = pandas.DataFrame(list_dataframes,columns=list_dataframes[0].keys())
df.to_csv("zomato_csvs/zomatoData"+".csv",index=False)